# Lab 1 - Creating the SQL Tables

In this lab, use `sqlalchemy` to create, populate, and query a table from the baseball database, as well as for the `super_hero_powers.csv` table.  

In [1]:
import pandas as pd
artwork = pd.read_csv("./data/Artworks.csv")

## Part 1 - Baseball Managers

In this part of the lab, you will walk through the process of creating a manager table from [Lahman’s Baseball Database](http://www.seanlahman.com/baseball-archive/statistics/)

## Task 1 - Download, unzip, rename 

1. Download the baseball database linked above (save to desktop)
2. Unzip the file and rename to `baseball`
3. Load the `core/Managers.csv` file into a pandas `DataFrame` using `read_csv`
4. Inspect the `column` names and `dtypes`

In [2]:
import pandas as pd
managers = pd.read_csv('/Users/bg8485bw/DSCI430/baseball/core/Managers.csv')
managers.head()

,playerID,yearID,teamID,lgID,inseason,G,W,L,rank,plyrMgr
0,wrighha01,1871,BS1,NaN,1,31,20,10,3.0,Y
1,woodji01,1871,CH1,NaN,1,28,19,9,2.0,Y
2,paborch01,1871,CL1,NaN,1,29,10,19,8.0,Y
3,lennobi01,1871,FW1,NaN,1,14,5,9,8.0,Y
4,deaneha01,1871,FW1,NaN,2,5,2,3,8.0,Y


In [3]:
managers.columns

Index(['playerID', 'yearID', 'teamID', 'lgID', 'inseason', 'G', 'W', 'L',
       'rank', 'plyrMgr'],
      dtype='object')

**Question:** Is there a candidate for a primary key?

In [4]:
[(col, managers[col].is_unique) for col in managers]

[('playerID', False),
 ('yearID', False),
 ('teamID', False),
 ('lgID', False),
 ('inseason', False),
 ('G', False),
 ('W', False),
 ('L', False),
 ('rank', False),
 ('plyrMgr', False)]

**Solution:** Add the `index` as an actual column

In [5]:
from dfply import mutate
managers = (managers >>
            mutate(id = managers.index))

In [6]:
managers.id.is_unique

True

In [7]:
managers.dtypes

playerID     object
yearID        int64
teamID       object
lgID         object
inseason      int64
G             int64
W             int64
L             int64
rank        float64
plyrMgr      object
id            int64
dtype: object

In [8]:
managers.shape

(3469, 11)

In [9]:
managers.head()

,playerID,yearID,teamID,lgID,inseason,G,W,L,rank,plyrMgr,id
0,wrighha01,1871,BS1,NaN,1,31,20,10,3.0,Y,0
1,woodji01,1871,CH1,NaN,1,28,19,9,2.0,Y,1
2,paborch01,1871,CL1,NaN,1,29,10,19,8.0,Y,2
3,lennobi01,1871,FW1,NaN,1,14,5,9,8.0,Y,3
4,deaneha01,1871,FW1,NaN,2,5,2,3,8.0,Y,4


#### Task 2 - Create a `sqlalchemy` types `dict`

In [10]:
from sqlalchemy import String, Integer
sql_types = {'id':Integer,
             'playerID':String, 
             'plyrMgr':String,
             'teamID':String, 
             'lgID':String, 
             'yearID':Integer, 
             'inseason':Integer, 
             'G':Integer, 
             'W':Integer, 
             'L':Integer,
             'rank':Integer}

#### Task 4 - Create an `engine` and `schema`

In [11]:
!rm databases/baseball.db #presentation purposes

In [12]:
from sqlalchemy import create_engine
mang_eng = create_engine("sqlite:///databases/baseball.db")
mang_eng.echo = True
schema = pd.io.sql.get_schema(managers, 'manager', keys='id', con=mang_eng, dtype=sql_types)
print(schema)


CREATE TABLE manager (
	"playerID" VARCHAR, 
	"yearID" INTEGER, 
	"teamID" VARCHAR, 
	"lgID" VARCHAR, 
	inseason INTEGER, 
	"G" INTEGER, 
	"W" INTEGER, 
	"L" INTEGER, 
	rank INTEGER, 
	"plyrMgr" VARCHAR, 
	id INTEGER NOT NULL, 
	CONSTRAINT manager_pk PRIMARY KEY (id)
)




#### Execute the `schema`

In [13]:
mang_eng.execute(schema)

2019-02-04 08:38:29,163 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-02-04 08:38:29,167 INFO sqlalchemy.engine.base.Engine ()
2019-02-04 08:38:29,170 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-02-04 08:38:29,173 INFO sqlalchemy.engine.base.Engine ()
2019-02-04 08:38:29,178 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE manager (
	"playerID" VARCHAR, 
	"yearID" INTEGER, 
	"teamID" VARCHAR, 
	"lgID" VARCHAR, 
	inseason INTEGER, 
	"G" INTEGER, 
	"W" INTEGER, 
	"L" INTEGER, 
	rank INTEGER, 
	"plyrMgr" VARCHAR, 
	id INTEGER NOT NULL, 
	CONSTRAINT manager_pk PRIMARY KEY (id)
)


2019-02-04 08:38:29,179 INFO sqlalchemy.engine.base.Engine ()
2019-02-04 08:38:29,185 INFO sqlalchemy.engine.base.Engine COMMIT


#### Task 5 - Use `to_sql` with `if_exists='append'` to insert the data

In [14]:
managers.to_sql('manager', 
                con=mang_eng, 
                dtype=sql_types, 
                index=False,
                if_exists='append')

2019-02-04 08:38:29,594 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("manager")
2019-02-04 08:38:29,600 INFO sqlalchemy.engine.base.Engine ()
2019-02-04 08:38:29,607 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-02-04 08:38:29,654 INFO sqlalchemy.engine.base.Engine INSERT INTO manager ("playerID", "yearID", "teamID", "lgID", inseason, "G", "W", "L", rank, "plyrMgr", id) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2019-02-04 08:38:29,656 INFO sqlalchemy.engine.base.Engine (('wrighha01', 1871, 'BS1', None, 1, 31, 20, 10, 3.0, 'Y', 0), ('woodji01', 1871, 'CH1', None, 1, 28, 19, 9, 2.0, 'Y', 1), ('paborch01', 1871, 'CL1', None, 1, 29, 10, 19, 8.0, 'Y', 2), ('lennobi01', 1871, 'FW1', None, 1, 14, 5, 9, 8.0, 'Y', 3), ('deaneha01', 1871, 'FW1', None, 2, 5, 2, 3, 8.0, 'Y', 4), ('fergubo01', 1871, 'NY2', None, 1, 33, 16, 17, 5.0, 'Y', 5), ('mcbridi01', 1871, 'PH1', None, 1, 28, 21, 7, 1.0, 'Y', 6), ('hastisc01', 1871, 'RC1', None, 1, 25, 4, 21, 9.0, 'Y', 7)  ... displaying 10 of

#### Task 6 - Query the table to make sure it all worked

In [15]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import select

mang_eng2 = create_engine("sqlite:///databases/baseball.db") 
Session = sessionmaker(mang_eng)
session = Session()

In [16]:
Base = automap_base()
Base.prepare(mang_eng2, reflect=True)
Manager = Base.classes.manager

In [17]:
from more_sqlalchemy import result_dicts
stmt = select('*').select_from(Manager)
session.execute(stmt).fetchmany(5) >> result_dicts

2019-02-04 08:38:30,493 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-02-04 08:38:30,495 INFO sqlalchemy.engine.base.Engine SELECT * 
FROM manager
2019-02-04 08:38:30,497 INFO sqlalchemy.engine.base.Engine ()


[{'G': 31,
  'L': 10,
  'W': 20,
  'id': 0,
  'inseason': 1,
  'lgID': None,
  'playerID': 'wrighha01',
  'plyrMgr': 'Y',
  'rank': 3,
  'teamID': 'BS1',
  'yearID': 1871},
 {'G': 28,
  'L': 9,
  'W': 19,
  'id': 1,
  'inseason': 1,
  'lgID': None,
  'playerID': 'woodji01',
  'plyrMgr': 'Y',
  'rank': 2,
  'teamID': 'CH1',
  'yearID': 1871},
 {'G': 29,
  'L': 19,
  'W': 10,
  'id': 2,
  'inseason': 1,
  'lgID': None,
  'playerID': 'paborch01',
  'plyrMgr': 'Y',
  'rank': 8,
  'teamID': 'CL1',
  'yearID': 1871},
 {'G': 14,
  'L': 9,
  'W': 5,
  'id': 3,
  'inseason': 1,
  'lgID': None,
  'playerID': 'lennobi01',
  'plyrMgr': 'Y',
  'rank': 8,
  'teamID': 'FW1',
  'yearID': 1871},
 {'G': 5,
  'L': 3,
  'W': 2,
  'id': 4,
  'inseason': 2,
  'lgID': None,
  'playerID': 'deaneha01',
  'plyrMgr': 'Y',
  'rank': 8,
  'teamID': 'FW1',
  'yearID': 1871}]

## Part 2 - Awards for Managers

Now add a table for the `AwardsManagers.csv` table.

In [18]:
awards_mang = pd.read_csv('/Users/bg8485bw/DSCI430/baseball/core/AwardsManagers.csv')
awards_mang.head()

,playerID,awardID,yearID,lgID,tie,notes
0,larusto01,BBWAA Manager of the Year,1983,AL,NaN,NaN
1,lasorto01,BBWAA Manager of the Year,1983,NL,NaN,NaN
2,andersp01,BBWAA Manager of the Year,1984,AL,NaN,NaN
3,freyji99,BBWAA Manager of the Year,1984,NL,NaN,NaN
4,coxbo01,BBWAA Manager of the Year,1985,AL,NaN,NaN


In [19]:
awards_mang.columns

Index(['playerID', 'awardID', 'yearID', 'lgID', 'tie', 'notes'], dtype='object')

In [20]:
[(col, awards_mang[col].is_unique) for col in awards_mang]

[('playerID', False),
 ('awardID', False),
 ('yearID', False),
 ('lgID', False),
 ('tie', False),
 ('notes', False)]

In [21]:
from dfply import mutate
awards_mang = (awards_mang >>
            mutate(id = awards_mang.index))

In [22]:
awards_mang.id.is_unique

True

In [23]:
awards_mang.dtypes

playerID    object
awardID     object
yearID       int64
lgID        object
tie         object
notes       object
id           int64
dtype: object

In [24]:
awards_mang.shape

(179, 7)

In [25]:
from sqlalchemy import String, Integer
sql_types = {'id' : Integer,
             'playerID': String, 
             'awardID' : String, 
             'yearID' : Integer,
             'lgID' : String,
             'tie' : String,
             'notes' : String,}

In [26]:
# Run schema
schema = pd.io.sql.get_schema(awards_mang, 'awards_manager', keys='id', con=mang_eng, dtype=sql_types)
print(schema)


CREATE TABLE awards_manager (
	"playerID" VARCHAR, 
	"awardID" VARCHAR, 
	"yearID" INTEGER, 
	"lgID" VARCHAR, 
	tie VARCHAR, 
	notes VARCHAR, 
	id INTEGER NOT NULL, 
	CONSTRAINT awards_manager_pk PRIMARY KEY (id)
)




In [27]:
mang_eng.execute(schema)

2019-02-04 08:38:33,351 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE awards_manager (
	"playerID" VARCHAR, 
	"awardID" VARCHAR, 
	"yearID" INTEGER, 
	"lgID" VARCHAR, 
	tie VARCHAR, 
	notes VARCHAR, 
	id INTEGER NOT NULL, 
	CONSTRAINT awards_manager_pk PRIMARY KEY (id)
)


2019-02-04 08:38:33,355 INFO sqlalchemy.engine.base.Engine ()
2019-02-04 08:38:33,360 INFO sqlalchemy.engine.base.Engine COMMIT


In [28]:
awards_mang.to_sql('awards_manager', 
                con=mang_eng, 
                dtype=sql_types, 
                index=False,
                if_exists='append')

2019-02-04 08:38:33,594 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("awards_manager")
2019-02-04 08:38:33,598 INFO sqlalchemy.engine.base.Engine ()
2019-02-04 08:38:33,606 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-02-04 08:38:33,613 INFO sqlalchemy.engine.base.Engine INSERT INTO awards_manager ("playerID", "awardID", "yearID", "lgID", tie, notes, id) VALUES (?, ?, ?, ?, ?, ?, ?)
2019-02-04 08:38:33,618 INFO sqlalchemy.engine.base.Engine (('larusto01', 'BBWAA Manager of the Year', 1983, 'AL', None, None, 0), ('lasorto01', 'BBWAA Manager of the Year', 1983, 'NL', None, None, 1), ('andersp01', 'BBWAA Manager of the Year', 1984, 'AL', None, None, 2), ('freyji99', 'BBWAA Manager of the Year', 1984, 'NL', None, None, 3), ('coxbo01', 'BBWAA Manager of the Year', 1985, 'AL', None, None, 4), ('herzowh01', 'BBWAA Manager of the Year', 1985, 'NL', None, None, 5), ('mcnamjo99', 'BBWAA Manager of the Year', 1986, 'AL', None, None, 6), ('lanieha01', 'BBWAA Manager of the Year

In [29]:
# Check if it works
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import select

mang_eng2 = create_engine("sqlite:///databases/baseball.db") 
Session = sessionmaker(mang_eng)
session = Session()

In [30]:
Base = automap_base()
Base.prepare(mang_eng2, reflect=True)
Awards_Manager = Base.classes.awards_manager

In [31]:
from more_sqlalchemy import result_dicts
stmt = select('*').select_from(Awards_Manager)
session.execute(stmt).fetchmany(5) >> result_dicts

2019-02-04 08:38:34,246 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-02-04 08:38:34,254 INFO sqlalchemy.engine.base.Engine SELECT * 
FROM awards_manager
2019-02-04 08:38:34,261 INFO sqlalchemy.engine.base.Engine ()


[{'awardID': 'BBWAA Manager of the Year',
  'id': 0,
  'lgID': 'AL',
  'notes': None,
  'playerID': 'larusto01',
  'tie': None,
  'yearID': 1983},
 {'awardID': 'BBWAA Manager of the Year',
  'id': 1,
  'lgID': 'NL',
  'notes': None,
  'playerID': 'lasorto01',
  'tie': None,
  'yearID': 1983},
 {'awardID': 'BBWAA Manager of the Year',
  'id': 2,
  'lgID': 'AL',
  'notes': None,
  'playerID': 'andersp01',
  'tie': None,
  'yearID': 1984},
 {'awardID': 'BBWAA Manager of the Year',
  'id': 3,
  'lgID': 'NL',
  'notes': None,
  'playerID': 'freyji99',
  'tie': None,
  'yearID': 1984},
 {'awardID': 'BBWAA Manager of the Year',
  'id': 4,
  'lgID': 'AL',
  'notes': None,
  'playerID': 'coxbo01',
  'tie': None,
  'yearID': 1985}]

In [32]:
stmt = select([Awards_Manager.playerID, Awards_Manager.yearID, Awards_Manager.lgID]).\
        where(Awards_Manager.lgID == 'NL')
results = session.execute(stmt).fetchmany(5)
results_dict = lambda results: list(map(lambda r: dict(zip(r.keys(), r)), results))
results_dict(session.execute(stmt).fetchmany(5))

2019-02-04 08:38:34,393 INFO sqlalchemy.engine.base.Engine SELECT awards_manager."playerID", awards_manager."yearID", awards_manager."lgID" 
FROM awards_manager 
WHERE awards_manager."lgID" = ?
2019-02-04 08:38:34,402 INFO sqlalchemy.engine.base.Engine ('NL',)
2019-02-04 08:38:34,410 INFO sqlalchemy.engine.base.Engine SELECT awards_manager."playerID", awards_manager."yearID", awards_manager."lgID" 
FROM awards_manager 
WHERE awards_manager."lgID" = ?
2019-02-04 08:38:34,417 INFO sqlalchemy.engine.base.Engine ('NL',)


[{'lgID': 'NL', 'playerID': 'lasorto01', 'yearID': 1983},
 {'lgID': 'NL', 'playerID': 'freyji99', 'yearID': 1984},
 {'lgID': 'NL', 'playerID': 'herzowh01', 'yearID': 1985},
 {'lgID': 'NL', 'playerID': 'lanieha01', 'yearID': 1986},
 {'lgID': 'NL', 'playerID': 'rodgebu01', 'yearID': 1987}]

In [33]:
from sqlalchemy import func
stmt3 = select([Awards_Manager.lgID, 
                func.count(Awards_Manager.lgID).label('lg(s)')]).\
          group_by(Awards_Manager.lgID)
session.execute(stmt3).fetchall() >> result_dicts

2019-02-04 08:38:34,443 INFO sqlalchemy.engine.base.Engine SELECT awards_manager."lgID", count(awards_manager."lgID") AS "lg(s)" 
FROM awards_manager GROUP BY awards_manager."lgID"
2019-02-04 08:38:34,452 INFO sqlalchemy.engine.base.Engine ()


[{'lg(s)': 65, 'lgID': 'AL'},
 {'lg(s)': 50, 'lgID': 'ML'},
 {'lg(s)': 64, 'lgID': 'NL'}]

## Part 2 - Super Hero Powers

Now make a database and table for the super hero powers.

## Problem 1
    
**Task:** One the `super_hero_powers.csv` and verify that the contents of the columns are all Boolean.  In this problem, you need to

1. Create a `dict` that defines the `pandas` column type
2. Read the file in using a `pd.read_csv`.
3. Clean up all the column labels.
    
**Be sure to write clean code!**


In [34]:
heroes = pd.read_csv('./data/super_hero_powers.csv')
heroes.head()

,hero_names,Agility,Accelerated Healing,Lantern Power Ring,Dimensional Awareness,Cold Resistance,Durability,Stealth,Energy Absorption,Flight,...,Web Creation,Reality Warping,Odin Force,Symbiote Costume,Speed Force,Phoenix Force,Molecular Dissipation,Vision - Cryo,Omnipresent,Omniscient
0,3-D Man,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,False,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,True,True,False,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [35]:
heroes.dtypes

hero_names               object
Agility                    bool
Accelerated Healing        bool
Lantern Power Ring         bool
Dimensional Awareness      bool
Cold Resistance            bool
Durability                 bool
Stealth                    bool
Energy Absorption          bool
Flight                     bool
Danger Sense               bool
Underwater breathing       bool
Marksmanship               bool
Weapons Master             bool
Power Augmentation         bool
Animal Attributes          bool
Longevity                  bool
Intelligence               bool
Super Strength             bool
Cryokinesis                bool
Telepathy                  bool
Energy Armor               bool
Energy Blasts              bool
Duplication                bool
Size Changing              bool
Density Control            bool
Stamina                    bool
Astral Travel              bool
Audio Control              bool
Dexterity                  bool
                          ...  
Intuitiv

In [36]:
heroes.shape

(667, 168)

In [37]:
heroes.columns

Index(['hero_names', 'Agility', 'Accelerated Healing', 'Lantern Power Ring',
       'Dimensional Awareness', 'Cold Resistance', 'Durability', 'Stealth',
       'Energy Absorption', 'Flight',
       ...
       'Web Creation', 'Reality Warping', 'Odin Force', 'Symbiote Costume',
       'Speed Force', 'Phoenix Force', 'Molecular Dissipation',
       'Vision - Cryo', 'Omnipresent', 'Omniscient'],
      dtype='object', length=168)

In [38]:
from dfply import *
clean_names = lambda name: name.strip().lower().replace(' ', '_').replace('-','').replace('__','_')
new_names = {clean_names(old_name):old_name for old_name in heroes.columns}
new_names

{'accelerated_healing': 'Accelerated Healing',
 'adaptation': 'Adaptation',
 'agility': 'Agility',
 'animal_attributes': 'Animal Attributes',
 'animal_control': 'Animal Control',
 'animal_oriented_powers': 'Animal Oriented Powers',
 'animation': 'Animation',
 'antigravity': 'Anti-Gravity',
 'astral_projection': 'Astral Projection',
 'astral_travel': 'Astral Travel',
 'audio_control': 'Audio Control',
 'banish': 'Banish',
 'biokinesis': 'Biokinesis',
 'camouflage': 'Camouflage',
 'changing_armor': 'Changing Armor',
 'clairvoyance': 'Clairvoyance',
 'cloaking': 'Cloaking',
 'cold_resistance': 'Cold Resistance',
 'cryokinesis': 'Cryokinesis',
 'danger_sense': 'Danger Sense',
 'darkforce_manipulation': 'Darkforce Manipulation',
 'death_touch': 'Death Touch',
 'density_control': 'Density Control',
 'dexterity': 'Dexterity',
 'dimensional_awareness': 'Dimensional Awareness',
 'dimensional_travel': 'Dimensional Travel',
 'duplication': 'Duplication',
 'durability': 'Durability',
 'echolocatio

In [39]:
from dfply import rename
heroes_renamed = (heroes >>
              rename(**new_names))
heroes_renamed.columns

Index(['hero_names', 'agility', 'accelerated_healing', 'lantern_power_ring',
       'dimensional_awareness', 'cold_resistance', 'durability', 'stealth',
       'energy_absorption', 'flight',
       ...
       'web_creation', 'reality_warping', 'odin_force', 'symbiote_costume',
       'speed_force', 'phoenix_force', 'molecular_dissipation', 'vision_cryo',
       'omnipresent', 'omniscient'],
      dtype='object', length=168)

In [40]:
[(col, heroes_renamed[col].is_unique) for col in heroes_renamed.columns]

[('hero_names', True),
 ('agility', False),
 ('accelerated_healing', False),
 ('lantern_power_ring', False),
 ('dimensional_awareness', False),
 ('cold_resistance', False),
 ('durability', False),
 ('stealth', False),
 ('energy_absorption', False),
 ('flight', False),
 ('danger_sense', False),
 ('underwater_breathing', False),
 ('marksmanship', False),
 ('weapons_master', False),
 ('power_augmentation', False),
 ('animal_attributes', False),
 ('longevity', False),
 ('intelligence', False),
 ('super_strength', False),
 ('cryokinesis', False),
 ('telepathy', False),
 ('energy_armor', False),
 ('energy_blasts', False),
 ('duplication', False),
 ('size_changing', False),
 ('density_control', False),
 ('stamina', False),
 ('astral_travel', False),
 ('audio_control', False),
 ('dexterity', False),
 ('omnitrix', False),
 ('super_speed', False),
 ('possession', False),
 ('animal_oriented_powers', False),
 ('weaponbased_powers', False),
 ('electrokinesis', False),
 ('darkforce_manipulation', Fa

In [41]:
list(heroes_renamed)
[]

[]

In [42]:
from sqlalchemy import String, Integer, Boolean
sql_types = {k:Boolean if k != 'hero_names' else String for k in heroes_renamed}
sql_types

{'accelerated_healing': sqlalchemy.sql.sqltypes.Boolean,
 'adaptation': sqlalchemy.sql.sqltypes.Boolean,
 'agility': sqlalchemy.sql.sqltypes.Boolean,
 'animal_attributes': sqlalchemy.sql.sqltypes.Boolean,
 'animal_control': sqlalchemy.sql.sqltypes.Boolean,
 'animal_oriented_powers': sqlalchemy.sql.sqltypes.Boolean,
 'animation': sqlalchemy.sql.sqltypes.Boolean,
 'antigravity': sqlalchemy.sql.sqltypes.Boolean,
 'astral_projection': sqlalchemy.sql.sqltypes.Boolean,
 'astral_travel': sqlalchemy.sql.sqltypes.Boolean,
 'audio_control': sqlalchemy.sql.sqltypes.Boolean,
 'banish': sqlalchemy.sql.sqltypes.Boolean,
 'biokinesis': sqlalchemy.sql.sqltypes.Boolean,
 'camouflage': sqlalchemy.sql.sqltypes.Boolean,
 'changing_armor': sqlalchemy.sql.sqltypes.Boolean,
 'clairvoyance': sqlalchemy.sql.sqltypes.Boolean,
 'cloaking': sqlalchemy.sql.sqltypes.Boolean,
 'cold_resistance': sqlalchemy.sql.sqltypes.Boolean,
 'cryokinesis': sqlalchemy.sql.sqltypes.Boolean,
 'danger_sense': sqlalchemy.sql.sqltypes

## Problem 2
    
Now define an `sqlalchemy` table for these data using `pandas` `to_sql` dataframe method.  You can use the `sqlalchemy.String` and `sqlalchemy.Boolean` columns type, which are [documented here](https://docs.sqlalchemy.org/en/latest/core/type_basics.html)

In [43]:
from sqlalchemy import create_engine
hero_eng = create_engine("sqlite:///databases/superheroes.db")
hero_eng.echo = True
schema = pd.io.sql.get_schema(heroes_renamed, 'superheroes', keys='hero_names', con=hero_eng, dtype=sql_types)
print(schema)


CREATE TABLE superheroes (
	hero_names VARCHAR NOT NULL, 
	agility BOOLEAN, 
	accelerated_healing BOOLEAN, 
	lantern_power_ring BOOLEAN, 
	dimensional_awareness BOOLEAN, 
	cold_resistance BOOLEAN, 
	durability BOOLEAN, 
	stealth BOOLEAN, 
	energy_absorption BOOLEAN, 
	flight BOOLEAN, 
	danger_sense BOOLEAN, 
	underwater_breathing BOOLEAN, 
	marksmanship BOOLEAN, 
	weapons_master BOOLEAN, 
	power_augmentation BOOLEAN, 
	animal_attributes BOOLEAN, 
	longevity BOOLEAN, 
	intelligence BOOLEAN, 
	super_strength BOOLEAN, 
	cryokinesis BOOLEAN, 
	telepathy BOOLEAN, 
	energy_armor BOOLEAN, 
	energy_blasts BOOLEAN, 
	duplication BOOLEAN, 
	size_changing BOOLEAN, 
	density_control BOOLEAN, 
	stamina BOOLEAN, 
	astral_travel BOOLEAN, 
	audio_control BOOLEAN, 
	dexterity BOOLEAN, 
	omnitrix BOOLEAN, 
	super_speed BOOLEAN, 
	possession BOOLEAN, 
	animal_oriented_powers BOOLEAN, 
	weaponbased_powers BOOLEAN, 
	electrokinesis BOOLEAN, 
	darkforce_manipulation BOOLEAN, 
	death_touch BOOLEAN, 
	telepo

In [44]:
hero_eng.execute(schema)

2019-02-04 08:38:35,718 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-02-04 08:38:35,720 INFO sqlalchemy.engine.base.Engine ()
2019-02-04 08:38:35,726 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-02-04 08:38:35,728 INFO sqlalchemy.engine.base.Engine ()
2019-02-04 08:38:35,731 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE superheroes (
	hero_names VARCHAR NOT NULL, 
	agility BOOLEAN, 
	accelerated_healing BOOLEAN, 
	lantern_power_ring BOOLEAN, 
	dimensional_awareness BOOLEAN, 
	cold_resistance BOOLEAN, 
	durability BOOLEAN, 
	stealth BOOLEAN, 
	energy_absorption BOOLEAN, 
	flight BOOLEAN, 
	danger_sense BOOLEAN, 
	underwater_breathing BOOLEAN, 
	marksmanship BOOLEAN, 
	weapons_master BOOLEAN, 
	power_augmentation BOOLEAN, 
	animal_attributes BOOLEAN, 
	longevity BOOLEAN, 
	intelligence BOOLEAN, 
	super_strength BOOLEAN, 
	cryokinesis BOOLEAN, 
	telepathy BOOLEAN, 
	energy_armo

2019-02-04 08:38:35,736 INFO sqlalchemy.engine.base.Engine ()
2019-02-04 08:38:35,741 INFO sqlalchemy.engine.base.Engine ROLLBACK


OperationalError: (sqlite3.OperationalError) table superheroes already exists [SQL: '\nCREATE TABLE superheroes (\n\thero_names VARCHAR NOT NULL, \n\tagility BOOLEAN, \n\taccelerated_healing BOOLEAN, \n\tlantern_power_ring BOOLEAN, \n\tdimensional_awareness BOOLEAN, \n\tcold_resistance BOOLEAN, \n\tdurability BOOLEAN, \n\tstealth BOOLEAN, \n\tenergy_absorption BOOLEAN, \n\tflight BOOLEAN, \n\tdanger_sense BOOLEAN, \n\tunderwater_breathing BOOLEAN, \n\tmarksmanship BOOLEAN, \n\tweapons_master BOOLEAN, \n\tpower_augmentation BOOLEAN, \n\tanimal_attributes BOOLEAN, \n\tlongevity BOOLEAN, \n\tintelligence BOOLEAN, \n\tsuper_strength BOOLEAN, \n\tcryokinesis BOOLEAN, \n\ttelepathy BOOLEAN, \n\tenergy_armor BOOLEAN, \n\tenergy_blasts BOOLEAN, \n\tduplication BOOLEAN, \n\tsize_changing BOOLEAN, \n\tdensity_control BOOLEAN, \n\tstamina BOOLEAN, \n\tastral_travel BOOLEAN, \n\taudio_control BOOLEAN, \n\tdexterity BOOLEAN, \n\tomnitrix BOOLEAN, \n\tsuper_speed BOOLEAN, \n\tpossession BOOLEAN, \n\

In [ ]:
heroes_renamed.to_sql('superheroes', 
                con=hero_eng, 
                dtype=sql_types, 
                index=False,
                if_exists='append')

## Problem 3
    
Now you need to make a new `engine`, `inspect` your database, and make a `session` to query the database.

In [54]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import select

hero_eng2 = create_engine("sqlite:///databases/superheroes.db") 
Session = sessionmaker(hero_eng)
session = Session()

In [55]:
Base = automap_base()
Base.prepare(hero_eng2, reflect=True)
superheroes = Base.classes.superheroes

In [56]:
from sqlalchemy import inspect
insp = inspect(hero_eng)

In [57]:
insp.get_table_names()

2019-02-04 08:38:48,833 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-02-04 08:38:48,837 INFO sqlalchemy.engine.base.Engine ()


['heroes', 'superheroes']

In [58]:
insp.get_columns('heroes')

2019-02-04 08:38:49,448 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("heroes")
2019-02-04 08:38:49,455 INFO sqlalchemy.engine.base.Engine ()


[{'autoincrement': 'auto',
  'default': None,
  'name': 'hero_names',
  'nullable': False,
  'primary_key': 1,
  'type': VARCHAR()},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'agility',
  'nullable': True,
  'primary_key': 0,
  'type': BOOLEAN()},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'accelerated_healing',
  'nullable': True,
  'primary_key': 0,
  'type': BOOLEAN()},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'lantern_power_ring',
  'nullable': True,
  'primary_key': 0,
  'type': BOOLEAN()},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'dimensional_awareness',
  'nullable': True,
  'primary_key': 0,
  'type': BOOLEAN()},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'cold_resistance',
  'nullable': True,
  'primary_key': 0,
  'type': BOOLEAN()},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'durability',
  'nullable': True,
  'primary_key': 0,
  'type': BOOLEAN()},
 {'autoincrement': 'auto',
  'default': None,

In [59]:
from more_sqlalchemy import result_dicts
stmt = select('*').select_from(superheroes)
session.execute(stmt).fetchmany(5) >> result_dicts

2019-02-04 08:38:50,014 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-02-04 08:38:50,019 INFO sqlalchemy.engine.base.Engine SELECT * 
FROM superheroes
2019-02-04 08:38:50,022 INFO sqlalchemy.engine.base.Engine ()


[{'accelerated_healing': 0,
  'adaptation': 0,
  'agility': 1,
  'animal_attributes': 0,
  'animal_control': 0,
  'animal_oriented_powers': 0,
  'animation': 0,
  'antigravity': 0,
  'astral_projection': 0,
  'astral_travel': 0,
  'audio_control': 0,
  'banish': 0,
  'biokinesis': 0,
  'camouflage': 0,
  'changing_armor': 0,
  'clairvoyance': 0,
  'cloaking': 0,
  'cold_resistance': 0,
  'cryokinesis': 0,
  'danger_sense': 0,
  'darkforce_manipulation': 0,
  'death_touch': 0,
  'density_control': 0,
  'dexterity': 0,
  'dimensional_awareness': 0,
  'dimensional_travel': 0,
  'duplication': 0,
  'durability': 0,
  'echolocation': 0,
  'elasticity': 0,
  'electrical_transport': 0,
  'electrokinesis': 0,
  'element_control': 0,
  'elemental_transmogrification': 0,
  'empathy': 0,
  'energy_absorption': 0,
  'energy_armor': 0,
  'energy_beams': 0,
  'energy_blasts': 0,
  'energy_constructs': 0,
  'energy_manipulation': 0,
  'energy_resistance': 0,
  'enhanced_hearing': 0,
  'enhanced_memor

## Problem 4
    
Perform `sqlalchemy` queries to answer each of the following questions.

1. How many heroes have both Super Strength and Super Speed?
2. How many heroes have names that start with the word *Black*
3. Are heroes with Agility more likely to have Stealth?
4. What fraction of all heroes that can fly also have Super Strength?
5. Consider heroes that have names that contain `"girl"`, `"boy"`, `"woman"`, or `"man"`.  Compute the following ratio

$$\frac{N(\text{boy or man})}{N(\text{girl or woman}}$$

**Hint:** You will need to use some combination of `where`, `group_by`, and `count` for each part.

In [60]:
from sqlalchemy import func
stmt = select([func.count(), superheroes.hero_names, superheroes.super_strength, superheroes.super_speed]).\
        where(superheroes.super_strength == True).\
        where(superheroes.super_speed == True)
results = session.execute(stmt).fetchmany(5)
results_dict = lambda results: list(map(lambda r: dict(zip(r.keys(), r)), results))
results_dict(session.execute(stmt).fetchmany(5))


2019-02-04 08:38:51,273 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1, superheroes.hero_names, superheroes.super_strength, superheroes.super_speed 
FROM superheroes 
WHERE superheroes.super_strength = 1 AND superheroes.super_speed = 1
2019-02-04 08:38:51,277 INFO sqlalchemy.engine.base.Engine ()
2019-02-04 08:38:51,284 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1, superheroes.hero_names, superheroes.super_strength, superheroes.super_speed 
FROM superheroes 
WHERE superheroes.super_strength = 1 AND superheroes.super_speed = 1
2019-02-04 08:38:51,287 INFO sqlalchemy.engine.base.Engine ()


[{'count_1': 219,
  'hero_names': 'Wonder Woman',
  'super_speed': True,
  'super_strength': True}]

In [61]:
stmt = select([func.count(), superheroes.hero_names]).\
        where(superheroes.hero_names.startswith("Black"))
        
results = session.execute(stmt).fetchmany(5)
results_dict = lambda results: list(map(lambda r: dict(zip(r.keys(), r)), results))
results_dict(session.execute(stmt).fetchmany(5))

2019-02-04 08:38:52,148 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1, superheroes.hero_names 
FROM superheroes 
WHERE (superheroes.hero_names LIKE ? || '%%')
2019-02-04 08:38:52,152 INFO sqlalchemy.engine.base.Engine ('Black',)
2019-02-04 08:38:52,158 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1, superheroes.hero_names 
FROM superheroes 
WHERE (superheroes.hero_names LIKE ? || '%%')
2019-02-04 08:38:52,160 INFO sqlalchemy.engine.base.Engine ('Black',)


[{'count_1': 16, 'hero_names': 'Blackwulf'}]

In [62]:
stmt = select([superheroes.agility, superheroes.stealth, 
               func.count(superheroes.stealth).label('stealth_count')]).\
where(superheroes.agility == True).\
group_by(superheroes.stealth)

results = session.execute(stmt).fetchmany(5)
results_dict = lambda results: list(map(lambda r: dict(zip(r.keys(), r)), results))
results_dict(session.execute(stmt).fetchmany(5))

2019-02-04 08:38:52,745 INFO sqlalchemy.engine.base.Engine SELECT superheroes.agility, superheroes.stealth, count(superheroes.stealth) AS stealth_count 
FROM superheroes 
WHERE superheroes.agility = 1 GROUP BY superheroes.stealth
2019-02-04 08:38:52,749 INFO sqlalchemy.engine.base.Engine ()
2019-02-04 08:38:52,753 INFO sqlalchemy.engine.base.Engine SELECT superheroes.agility, superheroes.stealth, count(superheroes.stealth) AS stealth_count 
FROM superheroes 
WHERE superheroes.agility = 1 GROUP BY superheroes.stealth
2019-02-04 08:38:52,755 INFO sqlalchemy.engine.base.Engine ()


[{'agility': True, 'stealth': False, 'stealth_count': 147},
 {'agility': True, 'stealth': True, 'stealth_count': 95}]

In [63]:
stmt = select([superheroes.flight, superheroes.super_strength, 
               func.count(superheroes.super_strength).label('super_strength_count')]).\
where(superheroes.flight == True).\
group_by(superheroes.super_strength)

results = session.execute(stmt).fetchmany(5)
results_dict = lambda results: list(map(lambda r: dict(zip(r.keys(), r)), results))
results_dict(session.execute(stmt).fetchmany(5))

2019-02-04 08:38:53,450 INFO sqlalchemy.engine.base.Engine SELECT superheroes.flight, superheroes.super_strength, count(superheroes.super_strength) AS super_strength_count 
FROM superheroes 
WHERE superheroes.flight = 1 GROUP BY superheroes.super_strength
2019-02-04 08:38:53,455 INFO sqlalchemy.engine.base.Engine ()
2019-02-04 08:38:53,461 INFO sqlalchemy.engine.base.Engine SELECT superheroes.flight, superheroes.super_strength, count(superheroes.super_strength) AS super_strength_count 
FROM superheroes 
WHERE superheroes.flight = 1 GROUP BY superheroes.super_strength
2019-02-04 08:38:53,465 INFO sqlalchemy.engine.base.Engine ()


[{'flight': True, 'super_strength': False, 'super_strength_count': 65},
 {'flight': True, 'super_strength': True, 'super_strength_count': 147}]

In [64]:
from sqlalchemy import or_
stmt = select([func.count(superheroes.hero_names)]).\
        where(or_(superheroes.hero_names.contains('boy'), superheroes.hero_names.contains('man')))
    
stmt1 = select([func.count(superheroes.hero_names)]).\
         where(or_(superheroes.hero_names.contains('girl'), superheroes.hero_names.contains('woman')))
      
results = session.execute(stmt).fetchone()
results1 = session.execute(stmt1).fetchone()
#results_dict = lambda results: list(map(lambda r: dict(zip(r.keys(), r)), results))
Male = results.values()
Female = results1.values()
R = zip(Male,Female)
Ratio = list(R)
Ratio = Ratio[0][0]/Ratio[0][1]
Ratio

2019-02-04 08:38:53,799 INFO sqlalchemy.engine.base.Engine SELECT count(superheroes.hero_names) AS count_1 
FROM superheroes 
WHERE (superheroes.hero_names LIKE '%%' || ? || '%%') OR (superheroes.hero_names LIKE '%%' || ? || '%%')
2019-02-04 08:38:53,802 INFO sqlalchemy.engine.base.Engine ('boy', 'man')
2019-02-04 08:38:53,806 INFO sqlalchemy.engine.base.Engine SELECT count(superheroes.hero_names) AS count_1 
FROM superheroes 
WHERE (superheroes.hero_names LIKE '%%' || ? || '%%') OR (superheroes.hero_names LIKE '%%' || ? || '%%')
2019-02-04 08:38:53,810 INFO sqlalchemy.engine.base.Engine ('girl', 'woman')


2.3846153846153846

## Problem 5

Tell me another cool fact about the super powers.

In [267]:
# Look Heroes that possess the most powers
#stmt = (select_sql('*').where(superheroes)
#results = session.execute(stmt).fetchall()
#results_dict = lambda results: list(map(lambda r: dict(zip(r.keys(), r)), results))
#results_dict(session.execute(stmt).fetchmany(2))      

In [268]:
# Obtain the number of powers each superhero has
Power_no = [(sum(row[True] == 1)) for row in heroes_renamed.iterrows()]
Power_no
Power_no1 = pd.DataFrame(Power_no)
Power_no1.rename(columns={0:'Number of Powers'}, inplace=True)
Power_no1.head()

,Number of Powers
0,4
1,7
2,16
3,1
4,8


In [269]:
# Shows which hero has the most powers
result = pd.concat([Power_no1, heroes_renamed], axis=1, sort=False)
result.sort_values('Number of Powers', ascending=False)

,Number of Powers,hero_names,agility,accelerated_healing,lantern_power_ring,dimensional_awareness,cold_resistance,durability,stealth,energy_absorption,...,web_creation,reality_warping,odin_force,symbiote_costume,speed_force,phoenix_force,molecular_dissipation,vision_cryo,omnipresent,omniscient
563,49,Spectre,True,True,False,True,False,False,True,True,...,False,True,False,False,False,False,False,False,False,False
18,44,Amazo,True,False,True,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
394,35,Martian Manhunter,True,False,False,False,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False
370,35,Living Tribunal,True,True,False,True,False,False,True,True,...,False,True,False,False,False,False,False,False,False,False
388,34,Man of Miracles,False,True,False,True,False,False,False,True,...,False,True,False,False,False,False,False,False,True,True
139,33,Captain Marvel,True,True,False,False,True,True,False,True,...,False,False,False,False,False,False,False,False,False,False
597,33,T-X,True,True,False,False,True,True,True,False,...,False,False,False,False,False,False,False,False,False,False
246,32,Galactus,False,True,False,True,False,True,False,True,...,False,False,False,False,False,False,False,False,False,False
594,32,T-1000,True,True,False,False,True,True,True,False,...,False,False,False,False,False,False,False,False,False,False
455,31,One-Above-All,True,True,False,True,False,True,False,True,...,False,True,False,False,False,False,False,False,True,True
